In [2]:
from kfp.v2.dsl import component, pipeline
from kfp.v2.dsl import Dataset, Output, Input, Metrics, Markdown, Artifact
from kfp.v2 import compiler

In [94]:
@component(
    base_image="gcr.io/ml-ops-segment-anything/sam:latest",
    packages_to_install=[
        'imantics',
        'scipy',
        'matplotlib'
    ]
)
def batch_prediction(
    bucket_name: str,
    input_dir: str,
    output_dir: str,
    prompt_json_file: str,
    max_mask: int,
    visualization: Output[Markdown],
):
    import torch
    from typing import Dict, List
    from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
    import base64
    import numpy as np
    import cv2
    import logging
    from google.cloud import storage
    import base64
    import json
    import time
    from imantics import Polygons, Mask
    from scipy.ndimage import median_filter
    import matplotlib.pyplot as plt

    def show_points(coords, labels, ax, marker_size=375):
        pos_points = coords[labels==1]
        neg_points = coords[labels==0]
        ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
        ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)  

    def show_mask(mask, ax, random_color=False):
        if random_color:
            color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
        else:
            color = np.array([30/255, 144/255, 255/255, 0.6])
        h, w = mask.shape[-2:]
        mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
        ax.imshow(mask_image)
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    # Get Model artifact from cloud storage
    blob = bucket.blob('model_artifacts/sam_vit_b_01ec64.pth')
    blob.download_to_filename('sam_vit_b_01ec64.pth')
    
    #Load the model
    sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
    print(torch.cuda.is_available())
    sam.to("cuda")
    # Automatic mask generation
    mask_generator = SamAutomaticMaskGenerator(sam)
    # SAM Masking with prompts
    predictor = SamPredictor(sam)
    
    # Initialize Images
    blobs = bucket.list_blobs(prefix=f"{input_dir}/")
    image_extensions = ('.png', '.jpg', '.jpeg')
    image_blobs = [blob for blob in blobs if blob.name.lower().endswith(image_extensions)]
    
    # Prompt Json
    blob = bucket.blob(f'{input_dir}/{prompt_json_file}')
    if blob.exists():
        prompt_json = json.loads(blob.download_as_text()) #returns prompts dictionary
    else:
        prompt_json = None
    print("prompt_json")
    print(prompt_json)

    # Reshaping the image
    def reshape_image(image, size=512):
        # Ratio for showing up in Markdown
        if image.shape[0] < size and image.shape[1] < size: 
            ratio = 1
        else: 
            ratio = size / max(image.shape[0], image.shape[1])
        width = int(image.shape[1] * ratio)
        height = int(image.shape[0] * ratio)
        image = cv2.resize(image, (width, height))
        return ratio, image

    # Predict all images
    start_time_pred = time.time()
    filenames, results = [], []
    for image_blob in image_blobs:
        filename = image_blob.name.split('/')[-1]
        filenames.append(filename)
        start_time = time.time()
        image_blob.download_to_filename(filename)
        print(f"Image Downloading: {time.time() - start_time}s")
        ratio, image = reshape_image(cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB))
        if prompt_json and filename in prompt_json: # Predicting with prompt inputs
            prompt_info = prompt_json[image_blob.name.split('/')[-1]]
            prompt_input, label_input = [], []
            for label, points in prompt_info.items():
                for point in points:
                    point = [int(c * ratio) for c in point]
                    prompt_input.append(point)
                    label_input.append(int(label))
            predictor.set_image(image)
            start_time = time.time()
            masks, scores, logits = predictor.predict(
                                        point_coords=np.array(prompt_input),
                                        point_labels=np.array(label_input),
                                        multimask_output=False
                                    )
            print(f"Prediction with Prompt: {time.time() - start_time}s")
            start_time = time.time()
            result = {
                "filename": filename,
                "image": image,
                "prediction_type" : "Predicting with Prompts",
                "prompt_input": prompt_input,
                "label_input": label_input,
                "ratio": ratio
            }
            result["masks"], result["polygon_vertices"] = {}, {}
            for idx, mask in enumerate(masks[:max_mask]):
                median_filter_size = int(min(mask.shape)//20)
                if median_filter_size % 2 == 0:
                    median_filter_size += 1
                mask = median_filter(mask, median_filter_size)
                result["masks"][f'mask_{idx}'] = mask
                polygons = Mask(mask).polygons()
                result["polygon_vertices"][f'mask_{idx}'] = list(map(lambda x: x.tolist(), polygons.points))
            print(f"Postprocessing {time.time() - start_time}s")
        else: # Predicting without prompt inputs
            start_time = time.time()
            masks = mask_generator.generate(image)
            print(f"Prediction without Prompt: {time.time() - start_time}s")
            result = {
                "filename": filename,
                "image": image,
                "prediction_type" : "Predicting without Prompts",
                "ratio": ratio
            }
            sorted_masks = sorted(masks, key=(lambda x: x['area']), reverse=True)
            result["masks"], result["polygon_vertices"] = {}, {}
            start_time = time.time()
            for idx, mask in enumerate(sorted(masks, key=(lambda x: x['area']), reverse=True)[:max_mask]):
                mask = mask['segmentation']
                median_filter_size = int(min(mask.shape)//20)
                if median_filter_size % 2 == 0:
                    median_filter_size += 1
                mask = median_filter(mask, median_filter_size)
                result["masks"][f'mask_{idx}'] = mask
                polygons = Mask(mask).polygons()
                result["polygon_vertices"][f'mask_{idx}'] = list(map(lambda x: x.tolist(), polygons.points))
            print(f"Postprocessing {time.time() - start_time}s")
        results.append(result)
        torch.cuda.empty_cache()
    print(f"Time Taken for predictions, including preprocessing and postprocessing: {time.time() - start_time_pred}s")

    # Export Images to Output DIR
    start_time = time.time()
    alpha = 0.5
    for filename, result in zip(filenames, results):
        original_image_blob = bucket.blob(f"{output_dir}/{filename}/original.jpg")
        original_image_blob.upload_from_filename(filename)
        for mask_name, mask in result["masks"].items():
            # mask_color = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
            # mask_color[mask == 0] = [0, 0, 0]
            # mask_color[mask == 1] = [255, 0, 0]
            # img_mask = result['image'].copy()
            # img_mask[mask==1, :] = (1-alpha) * img_mask[mask==1, :] + alpha * mask_color[mask==1, :3]
            # cv2.imwrite(f'{mask_name}.jpg', cv2.cvtColor(img_mask, cv2.COLOR_RGB2BGR))  
            mask = np.array(mask)
            plt.figure(figsize=(10,10))
            plt.imshow(result['image'])
            show_mask(mask, plt)
            if "prompt_input" in result:
                input_point = np.array(result['prompt_input'])
                input_label = np.array(result['label_input'])
                show_points(input_point, input_label, plt)
            plt.axis('off')
            plt.savefig(f"{mask_name}.jpg")
            plt.close()
            bucket.blob(f"{output_dir}/{filename}/{mask_name}.jpg").upload_from_filename(f"{mask_name}.jpg")
    print(f"Time Taken for image exportations: {time.time() - start_time}s")
    
    # Export Results to Output DIR
    start_time = time.time()
    for result in results:
        output_result = {}
        output_result["filename"] = result["filename"]
        output_result["prediction_type"] = result["prediction_type"]
        output_result["polygon_vertices"] = result["polygon_vertices"]
        output_result["ratio"] = result["ratio"]
        json_object = json.dumps(output_result, indent=4)
        with open("result.json", "w") as outfile:
            outfile.write(json_object)
        bucket.blob(f"{output_dir}/{result['filename']}/result.json").upload_from_filename("result.json")
    print(f"Time Taken for json exportations: {time.time() - start_time}s")
        
    # Visualization
    with open(visualization.path, 'w') as f:
        for result in results:
            image = result["image"]
            f.write(f"# {result['filename']} \n")
            f.write(f"## Prediction Type: {result['prediction_type']} \n")
            f.write("<table>")
            f.write("<tr><td>")
            f.write(f'<img src="https://storage.cloud.google.com/{bucket_name}/{output_dir}/{result["filename"]}/original.jpg">')
            f.write("</td>")
            f.write("<td></td><td></td><td></td></tr>")
            for idx, mask_name in enumerate(list(result["masks"].keys())):
                if idx % 4 == 0:
                    f.write("<tr>")
                f.write(f'<td><img src="https://storage.cloud.google.com/{bucket_name}/{output_dir}/{result["filename"]}/{mask_name}.jpg"></td>')
                if idx % 4 == 3:
                    f.write("</tr>")
            if idx % 4 != 3:
                f.write("</tr>")
            f.write("</table>\n\n")

In [98]:
# Pipeline Initialization
@pipeline(
    pipeline_root="gs://sam-pipeline-test",
    name="sam-pipeline-test-batch-100",
)
def sam_pipeline(
    bucket_name: str = "sam-pipeline-test",
    input_dir: str = "batch_10",
    output_dir: str = "batch_10_output",
    prompt_json_file: str = "dummy",
    max_mask: int = 10
):
    get_batch_prediction_op = (batch_prediction(
        bucket_name=bucket_name,
        input_dir=input_dir, 
        output_dir=output_dir,
        prompt_json_file=prompt_json_file,
        max_mask=max_mask
    )
        .set_cpu_limit("8")
        .set_memory_limit("64G")
        .add_node_selector_constraint("cloud.google.com/gke-accelerator", "NVIDIA_TESLA_T4")
        .set_gpu_limit(1)
    )

In [99]:
compiler.Compiler().compile(
    pipeline_func=sam_pipeline,
    package_path='sam_pipe_test.json')



In [100]:
!gsutil cp sam_pipe_test.json gs://sam-pipeline-test

Copying file://sam_pipe_test.json [Content-Type=application/json]...
/ [1 files][ 14.0 KiB/ 14.0 KiB]                                                
Operation completed over 1 objects/14.0 KiB.                                     


In [101]:
from google.cloud import aiplatform

job = aiplatform.PipelineJob(display_name = 'wallace_test',
                             template_path = 'sam_pipe_test.json',
                             enable_caching = False,
                             # failure_policy = "slow",
                             project="ml-ops-segment-anything",
                             location="us-central1",
                            )

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/633534855904/locations/us-central1/pipelineJobs/sam-pipeline-test-batch-100-20230728052635
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/633534855904/locations/us-central1/pipelineJobs/sam-pipeline-test-batch-100-20230728052635')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/sam-pipeline-test-batch-100-20230728052635?project=633534855904


In [6]:
!pip list | grep "kfp"

kfp                                    1.8.22
kfp-pipeline-spec                      0.1.16
kfp-server-api                         1.8.5


In [64]:
!pip install kfp==1.8.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kfp: filename=kfp-1.8.22-py3-none-any.whl size=426971 sha256=76abdae182cf74a0ba6b45e327f7bbd65044fa748967d5865e8d6bcf5aa660bb
  Stored in directory: /home/jupyter/.cache/pip/wheels/74/c0/fc/bf0ab209fd6ae814d7efbc821076e948c3e4884f846583ab58
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 1.8.20
    Uninstalling kfp-1.8.20:
      Successfully uninstalled kfp-1.8.20


In [17]:
!pip list | grep "kfp"

kfp                                    1.8.22
kfp-pipeline-spec                      0.1.16
kfp-server-api                         1.8.5
